In [17]:
import numpy as np
import pandas as pd
import math

In [37]:
house = pd.read_csv("house_prices.csv")

In [38]:
house

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


**Date sold column cleaning**

In [39]:
#since date column is encoded as datetime datatype we separate the year and the month it was sold dropping the time

house['date'] = pd.to_datetime(house['date'])
house['year_sold'] = house['date'].dt.year
house['month_sold'] = house['date'].dt.month
house['day_sold'] = house['date'].dt.day

house.drop(columns='date', inplace=True)


In [40]:
house

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year_sold,month_sold,day_sold
0,7129300520,221900.0,3,1.00,1180,5650,1.0,0,0,3,...,1955,0,98178,47.5112,-122.257,1340,5650,2014,10,13
1,6414100192,538000.0,3,2.25,2570,7242,2.0,0,0,3,...,1951,1991,98125,47.7210,-122.319,1690,7639,2014,12,9
2,5631500400,180000.0,2,1.00,770,10000,1.0,0,0,3,...,1933,0,98028,47.7379,-122.233,2720,8062,2015,2,25
3,2487200875,604000.0,4,3.00,1960,5000,1.0,0,0,5,...,1965,0,98136,47.5208,-122.393,1360,5000,2014,12,9
4,1954400510,510000.0,3,2.00,1680,8080,1.0,0,0,3,...,1987,0,98074,47.6168,-122.045,1800,7503,2015,2,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,360000.0,3,2.50,1530,1131,3.0,0,0,3,...,2009,0,98103,47.6993,-122.346,1530,1509,2014,5,21
21609,6600060120,400000.0,4,2.50,2310,5813,2.0,0,0,3,...,2014,0,98146,47.5107,-122.362,1830,7200,2015,2,23
21610,1523300141,402101.0,2,0.75,1020,1350,2.0,0,0,3,...,2009,0,98144,47.5944,-122.299,1020,2007,2014,6,23
21611,291310100,400000.0,3,2.50,1600,2388,2.0,0,0,3,...,2004,0,98027,47.5345,-122.069,1410,1287,2015,1,16


**Bathroom rounding up**

In [41]:
#we round up the numbers since there is no such thing as 0.75 of a bathroom that means it's just small 
house['num_bathrooms'] = house['bathrooms'].apply(lambda x: math.ceil(x))
house.drop(columns='bathrooms', inplace=True)

In [42]:
house

,id,price,bedrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year_sold,month_sold,day_sold,num_bathrooms
0,7129300520,221900.0,3,1180,5650,1.0,0,0,3,7,...,0,98178,47.5112,-122.257,1340,5650,2014,10,13,1
1,6414100192,538000.0,3,2570,7242,2.0,0,0,3,7,...,1991,98125,47.7210,-122.319,1690,7639,2014,12,9,3
2,5631500400,180000.0,2,770,10000,1.0,0,0,3,6,...,0,98028,47.7379,-122.233,2720,8062,2015,2,25,1
3,2487200875,604000.0,4,1960,5000,1.0,0,0,5,7,...,0,98136,47.5208,-122.393,1360,5000,2014,12,9,3
4,1954400510,510000.0,3,1680,8080,1.0,0,0,3,8,...,0,98074,47.6168,-122.045,1800,7503,2015,2,18,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,360000.0,3,1530,1131,3.0,0,0,3,8,...,0,98103,47.6993,-122.346,1530,1509,2014,5,21,3
21609,6600060120,400000.0,4,2310,5813,2.0,0,0,3,8,...,0,98146,47.5107,-122.362,1830,7200,2015,2,23,3
21610,1523300141,402101.0,2,1020,1350,2.0,0,0,3,7,...,0,98144,47.5944,-122.299,1020,2007,2014,6,23,1
21611,291310100,400000.0,3,1600,2388,2.0,0,0,3,8,...,0,98027,47.5345,-122.069,1410,1287,2015,1,16,3


**Year Renovated: Boolean Approach**

In [51]:
house['renovated'] = house['yr_renovated'].apply(lambda x: 0 if x == 0 else 1)

In [52]:
house

,id,price,bedrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode,lat,long,sqft_living15,sqft_lot15,year_sold,month_sold,day_sold,num_bathrooms,renovated
0,7129300520,221900.0,3,1180,5650,1.0,0,0,3,7,...,98178,47.5112,-122.257,1340,5650,2014,10,13,1,0
1,6414100192,538000.0,3,2570,7242,2.0,0,0,3,7,...,98125,47.7210,-122.319,1690,7639,2014,12,9,3,1
2,5631500400,180000.0,2,770,10000,1.0,0,0,3,6,...,98028,47.7379,-122.233,2720,8062,2015,2,25,1,0
3,2487200875,604000.0,4,1960,5000,1.0,0,0,5,7,...,98136,47.5208,-122.393,1360,5000,2014,12,9,3,0
4,1954400510,510000.0,3,1680,8080,1.0,0,0,3,8,...,98074,47.6168,-122.045,1800,7503,2015,2,18,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,360000.0,3,1530,1131,3.0,0,0,3,8,...,98103,47.6993,-122.346,1530,1509,2014,5,21,3,0
21609,6600060120,400000.0,4,2310,5813,2.0,0,0,3,8,...,98146,47.5107,-122.362,1830,7200,2015,2,23,3,0
21610,1523300141,402101.0,2,1020,1350,2.0,0,0,3,7,...,98144,47.5944,-122.299,1020,2007,2014,6,23,1,0
21611,291310100,400000.0,3,1600,2388,2.0,0,0,3,8,...,98027,47.5345,-122.069,1410,1287,2015,1,16,3,0


**Year Renovated: Categorical Approach**

In [53]:
house['renovation_status'] = house['yr_renovated'].apply(lambda x: 'Not Renovated' if x == 0 else str(x))




In [54]:
house

,id,price,bedrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,lat,long,sqft_living15,sqft_lot15,year_sold,month_sold,day_sold,num_bathrooms,renovated,renovation_status
0,7129300520,221900.0,3,1180,5650,1.0,0,0,3,7,...,47.5112,-122.257,1340,5650,2014,10,13,1,0,Not Renovated
1,6414100192,538000.0,3,2570,7242,2.0,0,0,3,7,...,47.7210,-122.319,1690,7639,2014,12,9,3,1,1991
2,5631500400,180000.0,2,770,10000,1.0,0,0,3,6,...,47.7379,-122.233,2720,8062,2015,2,25,1,0,Not Renovated
3,2487200875,604000.0,4,1960,5000,1.0,0,0,5,7,...,47.5208,-122.393,1360,5000,2014,12,9,3,0,Not Renovated
4,1954400510,510000.0,3,1680,8080,1.0,0,0,3,8,...,47.6168,-122.045,1800,7503,2015,2,18,2,0,Not Renovated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,360000.0,3,1530,1131,3.0,0,0,3,8,...,47.6993,-122.346,1530,1509,2014,5,21,3,0,Not Renovated
21609,6600060120,400000.0,4,2310,5813,2.0,0,0,3,8,...,47.5107,-122.362,1830,7200,2015,2,23,3,0,Not Renovated
21610,1523300141,402101.0,2,1020,1350,2.0,0,0,3,7,...,47.5944,-122.299,1020,2007,2014,6,23,1,0,Not Renovated
21611,291310100,400000.0,3,1600,2388,2.0,0,0,3,8,...,47.5345,-122.069,1410,1287,2015,1,16,3,0,Not Renovated


**Floors rounding up**

In [55]:
house['num_floors'] = house['floors'].apply(lambda x: math.ceil(x))
house.drop(columns='floors', inplace=True)

In [56]:
house

,id,price,bedrooms,sqft_living,sqft_lot,waterfront,view,condition,grade,sqft_above,...,long,sqft_living15,sqft_lot15,year_sold,month_sold,day_sold,num_bathrooms,renovated,renovation_status,num_floors
0,7129300520,221900.0,3,1180,5650,0,0,3,7,1180,...,-122.257,1340,5650,2014,10,13,1,0,Not Renovated,1
1,6414100192,538000.0,3,2570,7242,0,0,3,7,2170,...,-122.319,1690,7639,2014,12,9,3,1,1991,2
2,5631500400,180000.0,2,770,10000,0,0,3,6,770,...,-122.233,2720,8062,2015,2,25,1,0,Not Renovated,1
3,2487200875,604000.0,4,1960,5000,0,0,5,7,1050,...,-122.393,1360,5000,2014,12,9,3,0,Not Renovated,1
4,1954400510,510000.0,3,1680,8080,0,0,3,8,1680,...,-122.045,1800,7503,2015,2,18,2,0,Not Renovated,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,360000.0,3,1530,1131,0,0,3,8,1530,...,-122.346,1530,1509,2014,5,21,3,0,Not Renovated,3
21609,6600060120,400000.0,4,2310,5813,0,0,3,8,2310,...,-122.362,1830,7200,2015,2,23,3,0,Not Renovated,2
21610,1523300141,402101.0,2,1020,1350,0,0,3,7,1020,...,-122.299,1020,2007,2014,6,23,1,0,Not Renovated,2
21611,291310100,400000.0,3,1600,2388,0,0,3,8,1600,...,-122.069,1410,1287,2015,1,16,3,0,Not Renovated,2
